## <span style='color:#ff5f27'> 📝 Imports

In [1]:
from xgboost import XGBRegressor
import hopsworks
from functions.llm_chain import load_model, get_llm_chain, generate_response
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## <span style="color:#ff5f27;"> 🔮 Connect to Hopsworks Feature Store </span>

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/1150
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# Get_or_create the 'air_quality_fv' feature view
feature_view = fs.get_feature_view(
    name='air_quality_fv',
    version=1
)

# Initialize batch scoring
feature_view.init_batch_scoring(1)

weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

## <span style="color:#ff5f27;">🪝 Retrieve AirQuality Model from Model Registry</span>

In [5]:
# Retrieve the model registry
mr = project.get_model_registry()

# Retrieve the 'air_quality_xgboost_model' from the model registry
retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    version=1,
)

# Download the saved model artifacts  to a local directory
saved_model_dir = retrieved_model.download()

Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# model_air_quality = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
model_air_quality = XGBRegressor()

model_air_quality.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
model_air_quality

XGBRegressor(base_score='2.5461748E1', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float'], gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## <span style='color:#ff5f27'>⬇️ LLM Loading

In [7]:
import time
start_time = time.time()

# Load the LLM and its corresponding tokenizer.
model_llm, tokenizer = load_model()

duration = time.time() - start_time
print(f"The code execution took {duration} seconds.")

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading model from disk


`low_cpu_mem_usage` was None, now set to True since model is quantized.


The code execution took 11.343811511993408 seconds.


## <span style='color:#ff5f27'>⛓️ LangChain

In [8]:
import time
start_time = time.time()


# Create and configure a language model chain.
llm_chain = get_llm_chain(
    model_llm,
    tokenizer,
)

duration = time.time() - start_time
print(f"The code execution took {duration} seconds.")

The code execution took 0.4215388298034668 seconds.


## <span style='color:#ff5f27'>🧬 Model Inference


In [9]:
QUESTION7 = "Hi!"

response7 = generate_response(
    QUESTION7,
    feature_view,
    weather_fg,
    model_llm, 
    tokenizer,
    model_air_quality,
    llm_chain,
    verbose=False,
)

print(response7)

dict_keys(['context', 'date_today', 'question', 'chat_history', 'text'])


In [ ]:
QUESTION = "Who are you?"

response = generate_response(
    QUESTION,
    feature_view,
    weather_fg,
    model_llm,
    tokenizer,
    model_air_quality,
    llm_chain,
    verbose=False,
)

print(response)

In [14]:
QUESTION1 = "What was the average air quality from 2024-01-10 till 2024-01-14?"

response1 = generate_response(
    QUESTION1, 
    feature_view, 
    weather_fg,
    model_llm, 
    tokenizer, 
    model_air_quality, 
    llm_chain,
    verbose=False,
)

print(response1)

Finished: Reading data from Hopsworks, using ArrowFlight (0.82s) 

The average air quality from 2024-01-10 to 2024-01-14 was 10.6. This indicates that the air quality during that period was generally safe, but slightly above average.


In [ ]:
QUESTION11 = "When and what was the air quality like last week?"

response11 = generate_response(
    QUESTION11, 
    feature_view, 
    weather_fg,
    model_llm,
    tokenizer,
    model_air_quality,
    llm_chain,
    verbose=False,
)

print(response11)

In [ ]:
QUESTION12 = "When and what was the minimum air quality from 2024-01-10 till 2024-01-14?"

response12 = generate_response(
    QUESTION12, 
    feature_view, 
    weather_fg,
    model_llm, 
    tokenizer, 
    model_air_quality, 
    llm_chain,
    verbose=False,
)

print(response12)

In [ ]:
QUESTION2a = "What was the air quality like last week?"

response2 = generate_response(
    QUESTION2a,
    feature_view, 
    weather_fg,
    model_llm,
    tokenizer,
    model_air_quality,
    llm_chain,
    verbose=False,
)

print(response2)

In [ ]:
QUESTION2 = "What was the air quality like yesterday?"

response2 = generate_response(
    QUESTION2,
    feature_view, 
    weather_fg,
    model_llm,
    tokenizer,
    model_air_quality,
    llm_chain,
    verbose=False,
)

print(response2)

In [ ]:
QUESTION3 = "What will the air quality be like on 2024-03-20?"

response3 = generate_response(
    QUESTION3, 
    feature_view, 
    weather_fg,
    model_llm, 
    tokenizer,
    model_air_quality,
    llm_chain,
    verbose=False,
)

print(response3)

In [ ]:
QUESTION4 = "What will the air quality be like the day after tomorrow?"

response4 = generate_response(
    QUESTION4, 
    feature_view, 
    weather_fg,
    model_llm, 
    tokenizer, 
    model_air_quality, 
    llm_chain,
    verbose=False,
)

print(response4)

In [ ]:
QUESTION5 = "What will the air quality be like this Sunday?"

response5 = generate_response(
    QUESTION5, 
    feature_view, 
    weather_fg,
    model_llm, 
    tokenizer, 
    model_air_quality, 
    llm_chain,
    verbose=False,
)

print(response5)

In [ ]:
QUESTION7 = "What will the air quality be like for the rest of the week?"

response7 = generate_response(
    QUESTION7, 
    feature_view,
    weather_fg,
    model_llm,
    tokenizer, 
    model_air_quality, 
    llm_chain,
    verbose=False,
)

print(response7)

In [ ]:
QUESTION = "Will the air quality be safe or not for the next week?"

response = generate_response(
    QUESTION7, 
    feature_view, 
    weather_fg,
    model_llm, 
    tokenizer,
    model_air_quality,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION = "Is tomorrow's air quality level dangerous?"

response = generate_response(
    QUESTION, 
    feature_view, 
    weather_fg,
    model_llm, 
    tokenizer,
    model_air_quality, 
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION = "Can you please explain different air quality levels?"

response = generate_response(
    QUESTION, 
    feature_view, 
    weather_fg,
    model_llm, 
    tokenizer,
    model_air_quality, 
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np
import hopsworks
from xgboost import XGBRegressor
from functions.llm_chain import load_model, get_llm_chain, generate_response


In [ ]:
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    if y.ndim > 1 and y.shape[1] > 1:
        y = np.mean(y, axis=1)
    y /= np.max(np.abs(y))
    return transcriber({"sampling_rate": sr, "raw": y})["text"]

def generate_query_response(user_query):
    response = generate_response(
        user_query,
        feature_view,
        weather_fg,
        model_llm,
        tokenizer,
        model_air_quality,
        llm_chain,
        verbose=False,
    )
    return response

def handle_input(text_input=None, audio_input=None):
    if audio_input is not None:
        user_query = transcribe(audio_input)
    else:
        user_query = text_input
    
    if user_query:
        return generate_query_response(user_query)
    else:
        return "Please provide input either via text or voice."

iface = gr.Interface(
    fn=handle_input,
    inputs=[gr.Textbox(placeholder="Type here or use voice input..."), gr.Audio()],
    outputs="text",
    title="🌤️ AirQuality AI Assistant 💬",
    description="Ask your questions about air quality or use your voice to interact."
)

iface.launch(share=True)


---